In [73]:
import os
import random
import metpy
import datetime
import s3fs
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from pyresample import geometry, grid
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from skimage.filters import laplace
from skimage.filters import unsharp_mask
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

fs = s3fs.S3FileSystem(anon = True)
model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
random.seed(42)

In [76]:
def control(img):
    """
    Returns a dictionary of errors for the given file without sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to evaluate the error of
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    mae = mean_absolute_error(img, control)
    rmse = mean_squared_error(img, control, squared=False)
    return {'Control_MAE': mae, 'Control_RMSE': rmse}

def laplace_sharpening(img):
    """
    Returns a dictionary of errors for the given file using laplace sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + laplace_edges
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'LP_MAE': mae, 'LP_RMSE': rmse}

def unsharpmask_sharpening(img):
    """
    Returns a dictionary of errors for the given file using unsharpmask sharpening. To normalize images,
    each image is divided by its maximum value, and then multiplied by the same value after sharpening
    is completed.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a str
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'UM_MAE': mae, 'UM_RMSE': rmse}

def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    for x in os.listdir(path):
        if x[19:21] == band and file[23:37]==x[27:41]:
            return x

def update_dict(img, data):
    """
    Updates the dictionary with sharpening errors of a given file.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    
    Parameter data: Dictionary containing sharpening errors
    Precondition: data is a dict
    """
    newdata = {}
    newdata.update(control(img))
    newdata.update(laplace_sharpening(img))
    newdata.update(unsharpmask_sharpening(img))
    newdata.update(esrgan_sharpening(load_image(img)))
    data.update(newdata)

In [78]:
def load_image(array):
    """
    Returns loaded .npy file.
    
    Parameter path: Path to load .npy file from
    Precondition: path is a string
    """
    result = np.zeros((array.shape[0], array.shape[1], 3))
    result[:,:,0]= array
    result[:,:,1]= array
    result[:,:,2]= array
    result = result.astype('uint8')
    return result

def preprocess_image(array):
    """
    Returns preprocessed input array.
    
    Parameter array: array to preprocess
    Precondition: array is a numpy array
    """
    hr_image = array
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 2) * 2
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def downscale_image(image):
    """
    Returns low resolution image after scaling down input image using nearest neighbor downsampling.

    Parameter image: 3D of 4D tensor of preprocessed image
    Precondition: image is a tensor
    """
    image_size = []
    if len(image.shape) == 3:
        image_size = [image.shape[1], image.shape[0]]
    else:
        raise ValueError("Dimension mismatch. Can work only on single image.")

    image = tf.squeeze(tf.cast(image, tf.uint8))

    lr_image = np.asarray(Image.fromarray(image.numpy()).resize([image_size[0] // 2, image_size[1] // 2]))
    lr_image = tf.expand_dims(lr_image, 0)
    lr_image = tf.cast(lr_image, tf.float32)
    return lr_image

def esrgan_sharpening(image):
    """
    Returns dictionary containing errors after using the ESRGAN model.
    
    Parameter path: path of image not including file name
    Precondition: path is a string
    
    Parameter file: file of the image
    Precondition: file is a string of a .npy file
    """
    hr_image = preprocess_image(image)
    lr_image = downscale_image(tf.squeeze(hr_image))
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)
    
    hr_image = tf.squeeze(hr_image).numpy()[:,:,0]
    lr_image = tf.squeeze(tf.cast(lr_image, tf.uint8))
    lr_image = np.asarray(Image.fromarray(tf.squeeze(lr_image).numpy()).resize([hr_image.shape[0], hr_image.shape[1]]))[:,:,0]
    fake_image = resize(fake_image.numpy()[:,:,0], [hr_image.shape[0], hr_image.shape[1]])
    
    control_mae = mean_absolute_error(hr_image, lr_image)
    control_rmse = mean_squared_error(hr_image, lr_image, squared=False)

    mae = mean_absolute_error(hr_image, fake_image)
    rmse = mean_squared_error(hr_image, fake_image, squared=False)
    return {'Control_MAE*': control_mae, 'Control_RMSE*': control_rmse, 'SR_MAE': mae, 'SR_RMSE': rmse}

# Error

In [80]:
path = '../../GOES_Files/npy_files/'
pathCM = '../../GOES_Files/clear_sky_mask/'
data = []
dataCM = []

for i in range (20):
    CM = random.choice(os.listdir(pathCM))
    file07 = None
    file14 = None
    while file07 == None or file14 == None:
        file07 = find_npy_file(path, CM, '07')
        file14 = find_npy_file(path, CM, '14')
    
    data07 = {'File': file07, 'Band': file07[19:21]}
    img07 = np.load(path + file07)
    update_dict(img07, data07)
    data14 = {'File': file14, 'Band': file14[19:21]}
    img14 = np.load(path + file14)
    update_dict(img14, data14)
    
    datadiff = {'Band': 'diff'}
    diffimg = img07 - img14
    update_dict(diffimg, datadiff)

    data.append(data07)
    data.append(data14)
    data.append(datadiff)
    
    mask = np.load(pathCM + CM)
    mask = mask.astype(bool)
    img07[mask]=0
    img14[mask]=0
    
    data07CM = {'File': file07, 'Band': file07[19:21]}
    update_dict(img07, data07CM)
    data14CM = {'File': file14, 'Band': file14[19:21]}
    update_dict(img14, data14CM)

    datadiffCM = {'Band': 'diff'}
    diffimgCM = img07 - img14
    update_dict(diffimgCM, datadiffCM)

    dataCM.append(data07CM)
    dataCM.append(data14CM)
    dataCM.append(datadiffCM)
    
df = pd.DataFrame(data)
dfCM = pd.DataFrame(dataCM)

In [82]:
print("Mean Error")
df.groupby(df['Band']).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
                            'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'})

Mean Error


,Control_MAE,Control_RMSE,Control_MAE*,Control_RMSE*,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE,SR_MAE,SR_RMSE
Band,,,,,,,,,,
07,0.729880,1.073516,3.479306,11.536872,0.594339,0.893014,0.609034,0.903654,2.883735,8.028323
14,0.829373,1.288142,6.092629,18.168905,0.671194,1.069318,0.686614,1.083863,4.642838,12.694801
diff,0.638700,0.978273,2.029959,6.112104,0.526650,0.825121,0.543455,0.842156,1.829746,4.854568


In [83]:
print("Mean Error w/ CM")
dfCM.groupby(dfCM['Band']).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
                            'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'})

Mean Error w/ CM


,Control_MAE,Control_RMSE,Control_MAE*,Control_RMSE*,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE,SR_MAE,SR_RMSE
Band,,,,,,,,,,
07,19.775341,44.937721,1.682158,4.156406,17.001374,40.720667,17.196067,41.752071,1.706677,3.633067
14,19.579911,44.516488,1.634502,4.462536,16.833317,40.337395,17.016630,41.356551,1.665151,3.788337
diff,0.247675,0.539305,1.360866,4.893978,0.215738,0.487835,0.222710,0.501241,1.198370,3.926016


In [84]:
print("Standard Deviation of Error")
df.groupby(df['Band']).agg({'Control_MAE': 'std', 'Control_RMSE': 'std', 'Control_MAE*': 'std', 'Control_RMSE*': 'std',
                            'LP_MAE': 'std', 'LP_RMSE': 'std', 'UM_MAE': 'std', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'})

Standard Deviation of Error


,Control_MAE,Control_RMSE,Control_MAE*,Control_RMSE*,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE,SR_MAE,SR_RMSE
Band,,,,,,,,,,
07,0.729880,1.073516,3.479306,11.536872,0.594339,0.893014,0.609034,0.903654,2.883735,8.028323
14,0.829373,1.288142,6.092629,18.168905,0.671194,1.069318,0.686614,1.083863,4.642838,12.694801
diff,0.638700,0.978273,2.029959,6.112104,0.526650,0.825121,0.543455,0.842156,1.829746,4.854568


In [63]:
print("Standard Deviation of Error")
dfCM.groupby(dfCM['Band']).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
                            'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'})

Standard Deviation of Error


,Control_MAE,Control_RMSE,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE
Band,,,,,,
07,5.349255,9.266040,4.570101,8.365848,4.626849,8.593104
14,5.293796,9.173498,4.522336,8.281312,4.574776,8.505557
diff,0.071106,0.137732,0.061893,0.125321,0.064133,0.129041
